박수빈

In [7]:
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
df = pd.DataFrame(
    {
        'a': [1, 2, 3],
        'b': [4, 5, 6]
    }
)

In [6]:
index = np.random.randint(0, 10, size=5)
index

array([3, 2, 6, 3, 8])

In [8]:
a, b = train_test_split(range(10), test_size=0.3)

In [10]:
len(a), len(b)

(7, 3)

In [11]:
def f(a, **kwarg):
    return kwarg

f(1, b=1)

{'b': 1}

## PyTorch Syntax

In [ ]:
model = nn.Linear(in_features=1, out_features=1)

x_data = torch.Tensor([1, 2, 3, 4, 5]).view(-1, 1)
y_data = torch.Tensor([2, 4, 6, 8, 10]).view(-1, 1)

criterion = nn.MSELoss(model.parameters())
optimizer = torch.optim.Adam()

for _ in range(100):
    pred = model(x_data)
    loss = criterion(pred, y_data)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

[Parameter containing:
tensor([[-0.8512]], requires_grad=True), Parameter containing:
tensor([-0.5640], requires_grad=True)]
